# Build Unkown Dataset Notebook

## Purpose and Context

This notebook gets the experimental data we already have in a usable format for us to do predictions on

## Setup

import libraries

In [1]:
import pandas as pd
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import utils
import features

from rdkit import Chem

import tqdm.notebook
tqdm.notebook.tqdm_notebook.pandas()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_19908\3101613315.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


ModuleNotFoundError: No module named 'rdkit'

## Load Data

In [ ]:
data = utils.LoadDataFromOutput('extraction-pubChem')
data['Source'] = 'PubChem'
data.columns = data.columns.str.replace('_', ' ').str.title()
print(len(data))
data.head(1)

In [ ]:
data['Source Key'] = data['Cid'].astype(str)
data.rename(columns = {'Isosmiles': 'Smiles'}, inplace = True)
data = data[['Source', 'Source Key', 'Smiles']]

## Compute Features

In [2]:
temp = data['Smiles'].drop_duplicates().to_frame()

temp = temp.join(temp['Smiles'].progress_apply(features.ComputeAllFeatures).apply(lambda x: pd.Series(x, dtype = 'object'))).fillna(0)

# Removing any entry that failed to compute all features
temp = temp[temp['Total Atom Count'].isna() == False].drop_duplicates()

data = data.merge(temp, on = 'Smiles')

len(data)

NameError: name 'data' is not defined

Removing data that didn't calculate all the features

In [3]:
data = data[data['Error'] != True].reset_index(drop = True)
data.drop(['Error'], axis = 'columns', inplace = True)

NameError: name 'data' is not defined

Removing epsilon values we actually do know that are in one of the other datasets

In [4]:
knownEpsilons = utils.LoadDataFromOutput('dataset-allKnownEpsilon')['Smiles'].progress_apply(lambda x: Chem.inchi.MolToInchiKey(Chem.MolFromSmiles(x))).to_list()

data = data[data['InchiKey'].isin(knownEpsilons) == False].reset_index(drop = True)
len(data)

FileNotFoundError: [Errno 2] No such file or directory: '../output\\dataset-allKnownEpsilon.gzip.parquet'

### Cleaning up data

In [5]:
data.drop(['InchiKey'], axis = 'columns', inplace = True)

# Standardizing Column names
data.columns = data.columns.str.replace('_', ' ').str.title()

# Compressing data
utils.ConvertFloatColumnsToIntegerIfNoDataLoss(data)
utils.CompressIntegerColumns(data)

NameError: name 'data' is not defined

In [6]:
utils.RemoveStaticColumns(data)
print('-----------------')
print('-----------------')
print('-----------------')
utils.RemoveDuplicateColumns(data)

NameError: name 'data' is not defined

## Basic Analysis

In [7]:
data.info()

NameError: name 'data' is not defined

In [8]:
utils.InspectColumnValues(data)

NameError: name 'data' is not defined

In [9]:
data.describe()

NameError: name 'data' is not defined

In [ ]:
utils.ShowHistogramCharts(data)

### Saving for later use

In [ ]:
utils.SaveDataToOutput(data, 'dataset-unknownEpsilon')
utils.LoadDataFromOutput('dataset-unknownEpsilon')